# **Part 1: Integenerational Mobility Prediction: Data Cleaning**
* Date: 6/3/2023
* Group #: 6
* Names:
* Project: Empirical Project 3

## Importing Libraries

In [623]:
# Data Science Libraries
import numpy as np
import pandas as pd

# Machine Learning Libraries
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

## Q1 & Q2 are downloading, written information about it will be in the pdf file...

## 3. Reading in DataCommons dataset with pandas & Cleaning Dataset

In [624]:
df = pd.read_csv('United States of America_County.csv')

#### A. Separate out the geoid from the placedcid column

In [625]:
df['placeDcid'] = df['placeDcid'].str.replace('geoId/', '')

# Check
df.head(3)

,placeDcid,placeName,Date:Count_Person_Civilian_SingleMotherFamilyHousehold_NonInstitutionalized,Value:Count_Person_Civilian_SingleMotherFamilyHousehold_NonInstitutionalized,Source:Count_Person_Civilian_SingleMotherFamilyHousehold_NonInstitutionalized,Date:Count_Person_25OrMoreYears_EducationalAttainmentHighSchoolGraduateGedOrAlternative_NoHealthInsurance,Value:Count_Person_25OrMoreYears_EducationalAttainmentHighSchoolGraduateGedOrAlternative_NoHealthInsurance,Source:Count_Person_25OrMoreYears_EducationalAttainmentHighSchoolGraduateGedOrAlternative_NoHealthInsurance,Date:Percent_Person_Obesity,Value:Percent_Person_Obesity,Source:Percent_Person_Obesity,Date:Amount_EconomicActivity_GrossDomesticProduction_RealValue,Value:Amount_EconomicActivity_GrossDomesticProduction_RealValue,Source:Amount_EconomicActivity_GrossDomesticProduction_RealValue,Date:Count_HousingUnit,Value:Count_HousingUnit,Source:Count_HousingUnit
0,01001,Autauga County,2020.0,9465.0,https://data.census.gov/cedsci/table?q=S2701&t...,2014.0,2039.0,https://data.census.gov/cedsci/table?q=S2701&t...,2018.0,35.5,https://www.cdc.gov/places/index.html,2018.0,1.483414e+09,https://www.bea.gov/,2020.0,23697.0,https://www.census.gov/programs-surveys/acs/da...
1,17159,Richland County,2020.0,2052.0,https://data.census.gov/cedsci/table?q=S2701&t...,2014.0,398.0,https://data.census.gov/cedsci/table?q=S2701&t...,2018.0,34.3,https://www.cdc.gov/places/index.html,2018.0,9.622100e+08,https://www.bea.gov/,2020.0,7525.0,https://www.census.gov/programs-surveys/acs/da...
2,27035,Crow Wing County,2020.0,7021.0,https://data.census.gov/cedsci/table?q=S2701&t...,2014.0,1242.0,https://data.census.gov/cedsci/table?q=S2701&t...,2018.0,30.3,https://www.cdc.gov/places/index.html,2018.0,2.495344e+09,https://www.bea.gov/,2020.0,42326.0,https://www.census.gov/programs-surveys/acs/da...


#### B. Rename the column with the geoid data to geoid and change the datatype to int.

In [626]:
df['geoid'] = df['placeDcid']
df['place'] = df['placeName']
df.drop('placeDcid', axis=1, inplace=True)
df.drop('placeName', axis=1, inplace=True)

# Check and change datatype
df['geoid'] = df['geoid'].astype(int)

#### C. Remove the source* and date* columns from the dataset. 

In [627]:
columns_to_drop = df.columns[df.columns.str.contains('Source:|Date:')]
df.drop(columns_to_drop, axis=1, inplace=True)

# Check
df.head(3)

,Value:Count_Person_Civilian_SingleMotherFamilyHousehold_NonInstitutionalized,Value:Count_Person_25OrMoreYears_EducationalAttainmentHighSchoolGraduateGedOrAlternative_NoHealthInsurance,Value:Percent_Person_Obesity,Value:Amount_EconomicActivity_GrossDomesticProduction_RealValue,Value:Count_HousingUnit,geoid,place
0,9465.0,2039.0,35.5,1.483414e+09,23697.0,1001,Autauga County
1,2052.0,398.0,34.3,9.622100e+08,7525.0,17159,Richland County
2,7021.0,1242.0,30.3,2.495344e+09,42326.0,27035,Crow Wing County


#### D. Remove the value in front of your 5 predictors.

In [628]:
df.columns = df.columns.str.replace('Value:', '')

#### E. . Drop variables with no observations.

In [629]:
print(df.shape)
b4 = df.shape[0]

df.dropna(inplace=True)
print(df.shape)
after = df.shape[0]

print("Dropped:", b4-after, "Columns")

(3222, 7)
(3087, 7)
Dropped: 135 Columns


#### F. Reorder the variable using the code provided in the template.

In [630]:
df.columns

new_column_order = ['geoid', 
                    'place', 
                    'Count_Person_Civilian_SingleMotherFamilyHousehold_NonInstitutionalized',
                    'Count_Person_25OrMoreYears_EducationalAttainmentHighSchoolGraduateGedOrAlternative_NoHealthInsurance',
                    'Percent_Person_Obesity',
                    'Amount_EconomicActivity_GrossDomesticProduction_RealValue']

## 4. Merging dataset with .dta atlas_training data file.

In [631]:
atlas_df = pd.read_stata('atlas_training.dta')
atlas_df.head(5)

,geoid,place,pop,housing,kfr_pooled_p25,test,training,P_1,P_2,P_3,...,P_112,P_113,P_114,P_115,P_116,P_117,P_118,P_119,P_120,P_121
0,1003.0,Baldwin County,187114,104061,0.388847,0.0,1.0,82.847946,98.593452,101.776711,...,0.0,19.100000,0.00,2.01,0.00,3.52,0.0,0.00,0.0,0.0
1,1005.0,Barbour County,27321,11829,0.349386,0.0,1.0,76.313896,93.878723,90.702942,...,0.0,45.160000,0.00,4.84,0.00,1.61,0.0,3.23,0.0,0.0
2,1007.0,Bibb County,22754,8981,0.363391,0.0,1.0,73.765617,104.868469,82.129547,...,0.0,30.910000,0.00,7.27,0.00,1.82,0.0,0.00,0.0,0.0
3,1013.0,Butler County,20624,9964,0.357249,0.0,1.0,92.096672,121.073296,117.823196,...,0.0,41.070000,0.00,5.36,0.00,1.79,0.0,0.00,0.0,0.0
4,1015.0,Calhoun County,117714,53289,0.361847,0.0,1.0,76.938210,95.478249,98.326622,...,0.0,18.790001,0.61,3.03,0.61,2.42,0.0,0.00,0.0,0.0


In [632]:
# Merge Column based off of geoid column
df = pd.merge(df, atlas_df, on='geoid', how='outer')

## 5. counts --> rates

*Side, changing variable names for better readability of code.*


In [633]:
# Changing some variable names so they are easier to read
df['single-mom-persons'] = df['Count_Person_Civilian_SingleMotherFamilyHousehold_NonInstitutionalized']
df['HS-Grad-persons'] = df['Count_Person_25OrMoreYears_EducationalAttainmentHighSchoolGraduateGedOrAlternative_NoHealthInsurance']
df['Total-GDP'] = df['Amount_EconomicActivity_GrossDomesticProduction_RealValue']
df.drop(['Count_Person_Civilian_SingleMotherFamilyHousehold_NonInstitutionalized',
         'Count_Person_25OrMoreYears_EducationalAttainmentHighSchoolGraduateGedOrAlternative_NoHealthInsurance',
         'Amount_EconomicActivity_GrossDomesticProduction_RealValue'],
         axis=1,
         inplace=True)

Converting Counts to rates for each varaible

In [634]:
df.dropna(inplace=True)

In [635]:
df['single-mom-perc'] = df['single-mom-persons'] / df['pop']
df['HS-Grad-perc'] = df['HS-Grad-persons'] / df['pop']
df['GDP-per-capita'] = df['Total-GDP'] / df['pop']
df['Housing-units-per-capita'] = df['Count_HousingUnit']  / df['pop']

In [636]:
df.drop(['single-mom-persons', 
         'HS-Grad-persons', 
         'Count_HousingUnit',
         'Total-GDP'],
         axis=1,
         inplace=True)

## 6. Produce simple summary Statistics for 5 predictors and krf_pooled_p25

In [637]:
df[['single-mom-perc', 'HS-Grad-perc', 'GDP-per-capita', 'Housing-units-per-capita', 'Percent_Person_Obesity', 'kfr_pooled_p25']].describe()

,single-mom-perc,HS-Grad-perc,GDP-per-capita,Housing-units-per-capita,Percent_Person_Obesity,kfr_pooled_p25
count,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000
mean,0.151483,0.038659,44159.283869,0.472486,35.092077,0.416314
std,0.052313,0.014413,46962.472469,0.083343,4.484188,0.048736
min,0.037298,0.006348,7913.706248,0.274010,16.800000,0.292945
25%,0.115620,0.028202,27215.357529,0.432003,32.500000,0.381349
50%,0.142841,0.038056,36525.295871,0.456020,35.400000,0.412624
75%,0.179189,0.047517,50302.828314,0.487540,38.000000,0.443145
max,0.400866,0.104853,919531.905626,1.222418,50.100000,0.614030


#### 7. Run a linear regression of krf_pooled_p25 on the predictors you chose (converted to rates when appropriate) from DataCommons, in addition to the predictors already in the training dataset inspect the results, and comment on what you find. 

In [638]:
dependent_cols = [
 'Percent_Person_Obesity',
 'single-mom-perc',
 'HS-Grad-perc',
 'GDP-per-capita',
 'Housing-units-per-capita',
 'P_1',
 'P_2',
 'P_3',
 'P_4',
 'P_5',
 'P_6',
 'P_7',
 'P_8',
 'P_9',
 'P_10',
 'P_11',
 'P_12',
 'P_13',
 'P_14',
 'P_15',
 'P_16',
 'P_17',
 'P_18',
 'P_19',
 'P_20',
 'P_21',
 'P_22',
 'P_23',
 'P_24',
 'P_25',
 'P_26',
 'P_27',
 'P_28',
 'P_29',
 'P_30',
 'P_31',
 'P_32',
 'P_33',
 'P_34',
 'P_35',
 'P_36',
 'P_37',
 'P_38',
 'P_39',
 'P_40',
 'P_41',
 'P_42',
 'P_43',
 'P_44',
 'P_45',
 'P_46',
 'P_47',
 'P_48',
 'P_49',
 'P_50',
 'P_51',
 'P_52',
 'P_53',
 'P_54',
 'P_55',
 'P_56',
 'P_57',
 'P_58',
 'P_59',
 'P_60',
 'P_61',
 'P_62',
 'P_63',
 'P_64',
 'P_65',
 'P_66',
 'P_67',
 'P_68',
 'P_69',
 'P_70',
 'P_71',
 'P_72',
 'P_73',
 'P_74',
 'P_75',
 'P_76',
 'P_77',
 'P_78',
 'P_79',
 'P_80',
 'P_81',
 'P_82',
 'P_83',
 'P_84',
 'P_85',
 'P_86',
 'P_87',
 'P_88',
 'P_89',
 'P_90',
 'P_91',
 'P_92',
 'P_93',
 'P_94',
 'P_95',
 'P_96',
 'P_97',
 'P_98',
 'P_99',
 'P_100',
 'P_101',
 'P_102',
 'P_103',
 'P_104',
 'P_105',
 'P_106',
 'P_107',
 'P_108',
 'P_109',
 'P_110',
 'P_111',
 'P_112',
 'P_113',
 'P_114',
 'P_115',
 'P_116',
 'P_117',
 'P_118',
 'P_119',
 'P_120',
 'P_121'
]

In [639]:
X = df[dependent_cols]
y = df.kfr_pooled_p25

# Create and fit the linear regression model
mX = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X)
results = model.fit()

# Get coefficients and p-values
print(results)
intercept = results.params[0]
coefficients = results.params
p_values = results.pvalues

In [640]:
analysis = pd.concat([coefficients, p_values], axis=1)
analysis.columns = ["Coefficient", "P-Values"]

In [641]:
analysis['abs_Coefficient'] = analysis.Coefficient.abs()
analysis.sort_values('abs_Coefficient', ascending=False).head(10)

,Coefficient,P-Values,abs_Coefficient
P_40,0.389779,1.188703e-01,0.389779
P_38,0.295414,5.504781e-02,0.295414
P_39,0.236872,1.344962e-01,0.236872
P_35,-0.227715,1.213062e-01,0.227715
P_45,-0.220113,3.672697e-10,0.220113
P_50,0.215914,1.354919e-01,0.215914
HS-Grad-perc,0.206756,6.330823e-03,0.206756
P_36,-0.183642,4.066025e-01,0.183642
P_37,0.162781,2.978253e-01,0.162781
P_32,0.161018,4.296203e-05,0.161018


Took the dependent variables name and the coefficient's absolute value. The most correllated value is 
1. Share Asian in 2000
2. Share White 2000
3. Share hisp 2000
4. Share hisp 2010
5. Share of Single Headed Households with children 2000. 
6. Share of people who are not white
7. High School Graduate percentage
8. Share asian 2010
9. share black 2000
10. Share Below Poverty Line 2000 

#### 8. How well does your linear regression predict krf_pooled_p25 in-sample? You have to calculate the mean squared error.

In [643]:
y_pred = results.predict(X)

# Compute mean squared error
mse = mean_squared_error(y, y_pred)

print('Mean Squared Error:', mse)

Mean Squared Error: 0.00030053713818889265
